# Bedrock Agents as Tools with Strands Agents
Strands agents and Amazon Bedrock agents can seamlessly collaborate within a multi-agent system. By wrapping an existing Bedrock agent as a tool, a new Strands agent can invoke and interact with the Bedrock agent—enabling powerful workflows without needing to rewrite or modify the original Bedrock agent. This approach allows you to combine the strengths of both platforms, orchestrating tasks and sharing information between agents through well-defined interfaces.

In this lab, you'll learn how to invoke an Amazon Bedrock agent from a Strands agent by wrapping the Bedrock agent as a tool. This approach enables seamless integration of external AI services into your multi-agent workflows using the "agents as tools" pattern.

"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.



## Key Benefits and Core Principles

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task


In [4]:
!pip install -r requirements.txt

In [12]:
import logging
import boto3
import json
from botocore.exceptions import ClientError
from typing import Dict, Any
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class BedrockAgentManager:
    def __init__(self, region_name='us-west-2'):
        """
        Initialize the BedrockAgentManager with a boto3 client.
        
        :param region_name: AWS region name (default: us-west-2)
        """
        self.client = boto3.client('bedrock-agent', region_name=region_name)
        self.agent_id = None
        self.logger = logging.getLogger(__name__)
        
    def create_research_agent(
            self,
            agent_name: str,
            role_arn: str,
            foundation_model: str = "anthropic.claude-v2"
        ) -> Dict[str, Any]:
            """
            Creates a research agent using Amazon Bedrock.
            
            :param agent_name: Name for the research agent
            :param role_arn: ARN of the IAM role with necessary permissions
            :param foundation_model: The foundation model to be used (default: Claude v2)
            :return: The created agent's details if successful
            """
            # Define the research assistant instructions
            instruction = """
            You are a research assistant specialized in gathering and analyzing information.
            
            Your responsibilities include:
            1. Conducting thorough research on given topics
            2. Providing comprehensive, factual information
            3. Structuring responses with clear sections
            4. Including citations and sources where possible
            5. Highlighting key findings and insights
            
            When responding to queries:
            - Start with a brief overview
            - Break down complex topics into digestible sections
            - Use bullet points for key information
            - Cite sources when providing specific facts
            - Conclude with key takeaways
            
            Always maintain:
            - Accuracy in information
            - Objectivity in analysis
            - Clear organization of content
            - Professional tone
            """
            
            try:
                # Create the agent
                response = self.client.create_agent(
                    agentName=agent_name,
                    foundationModel=foundation_model,
                    agentResourceRoleArn=role_arn,
                    instruction=instruction,
                )
                
                self.agent_id = response["agent"]["agentId"]
                self.logger.info(f"Successfully created research agent: {agent_name}")
                self.logger.info(f"Bedrock Agent ID: {self.agent_id}")
                import os
                os.environ["BEDROCK_AGENT_ID"] = self.agent_id
                self.logger.info("Exported Bedrock Agent ID as environment variable BEDROCK_AGENT_ID")
                return response["agent"]
                
            except ClientError as e:
                self.logger.error(f"Error: Couldn't create research agent. Here's why: {e}")
                raise


In [13]:
# Initialize the BedrockAgentManager
agent_manager = BedrockAgentManager()

import boto3
import json

# Get the ARN of the currently active role from the credentials
sts_client = boto3.client('sts')
caller_identity = sts_client.get_caller_identity()
current_role_arn = None

# Try to infer the role ARN from the caller identity (works if using assumed role)
arn = caller_identity.get("Arn", "")
if ":assumed-role/" in arn:
    # arn:aws:sts::<account-id>:assumed-role/<role-name>/<session-name>
    # Convert to IAM role ARN: arn:aws:iam::<account-id>:role/<role-name>
    parts = arn.split(":assumed-role/")
    account_id = caller_identity["Account"]
    role_name = parts[1].split("/")[0]
    current_role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
else:
    # Fallback: use the full ARN (may be a user, not a role)
    current_role_arn = arn

print(f"Current principal ARN for delegation: {current_role_arn}")

# Create the required IAM role for Bedrock Agent
iam_client = boto3.client('iam')

role_name = "BedrockAgentRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": current_role_arn
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
        Description="Role for Bedrock Agent with required permissions"
    )
    print(f"IAM role '{role_name}' created successfully.")
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"IAM role '{role_name}' already exists.")

# Attach the required policy to the role (replace with actual policy ARN as needed)
policy_arn = "arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
try:
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn=policy_arn
    )
    print(f"Policy '{policy_arn}' attached to role '{role_name}'.")
except Exception as e:
    print(f"Failed to attach policy: {e}")

# Get the role ARN
role = iam_client.get_role(RoleName=role_name)
role_arn = role['Role']['Arn']
print(f"Using IAM Role ARN: {role_arn}")

try:
    # Create the research agent
    agent = agent_manager.create_research_agent(
        agent_name="ResearchAssistant",
        role_arn=role_arn
    )
    print("Research agent created successfully!")
    print(f"Agent ID: {agent_manager.agent_id}")
except Exception as e:
    print(f"Failed to create research agent: {str(e)}")


Current principal ARN for delegation: arn:aws:iam::334734167946:user/computer-use
IAM role 'BedrockAgentRole' already exists.
Policy 'arn:aws:iam::aws:policy/AmazonBedrockFullAccess' attached to role 'BedrockAgentRole'.
Using IAM Role ARN: arn:aws:iam::334734167946:role/BedrockAgentRole


INFO:__main__:Successfully created research agent: ResearchAssistant
INFO:__main__:Bedrock Agent ID: 1YUJ2BQP1I
INFO:__main__:Exported Bedrock Agent ID as environment variable BEDROCK_AGENT_ID


Research agent created successfully!
Agent ID: 1YUJ2BQP1I


In this module we will be creating an orchestrator based multi-agent workflow. 

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>


## Research Agent

Lets create strands bedrock agent as tool

### Orchestrator Agent

In [17]:
import os
from strands import Agent, tool
from strands_tools import file_write
import boto3

@tool(name="bedrock_research_assistant")
def bedrock_research_assistant(query: str) -> str:
    """Process and respond to research-related queries using Amazon Bedrock Agent."""
    try:
        # Initialize Bedrock Agent Runtime client
        bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-west-2')
        
        # Get agent ID from environment
        agent_id = os.environ.get("BEDROCK_AGENT_ID")
        if not agent_id:
            raise ValueError("BEDROCK_AGENT_ID environment variable is not set.")

        # Create a session
        session_response = bedrock_agent_runtime.create_session(agentId=agent_id)
        session_id = session_response['sessionId']
        
        # Invoke the agent
        response = bedrock_agent_runtime.invoke_agent(
            agentId=agent_id,
            sessionId=session_id,
            input={'text': query}
        )
        
        # Close the session
        bedrock_agent_runtime.delete_session(agentId=agent_id, sessionId=session_id)
        
        return response['completion']
    except Exception as e:
        return f"Error in Bedrock research assistant: {str(e)}"


In [18]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the bedrock_research_assistant tool
- For product recommendations and shopping advice → Answer directly
- For travel planning and itineraries → Answer directly
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [19]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        bedrock_research_assistant,
        file_write,
    ],
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [20]:
!pip install langfuse

In [ ]:
from langfuse import Langfuse

# Get keys for your project from the project settings page: https://cloud.langfuse.com
public_key = "pk"
secret_key = "sk"

os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
#os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Set up endpoint
otel_endpoint = str(os.environ.get("LANGFUSE_HOST")) + "/api/public/otel/v1/traces"

# Create authentication token:
import base64
auth_token = base64.b64encode(f"{public_key}:{secret_key}".encode()).decode()
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = otel_endpoint
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {auth_token}"

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host="https://cloud.langfuse.com"
)

In [23]:
# Example: E-commerce Customer Service System
customer_query = (
    "analyze how should travel recommendation be determined."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

I'll help you with hiking boot recommendations and then write the response to a file in the current directory.

## Hiking Boot Recommendations

When choosing hiking boots, consider these key factors:

### **Top-Rated Hiking Boot Brands:**
- **Merrell** - Great for beginners, comfortable and affordable
- **Salomon** - Excellent for technical terrain and durability
- **Keen** - Known for toe protection and wide fit options
- **Vasque** - Superior support for heavy loads
- **La Sportiva** - Premium quality for serious hikers

### **Key Features to Look For:**
1. **Waterproofing** - Gore-Tex or similar membrane
2. **Ankle Support** - Mid or high-cut for stability
3. **Sole Grip** - Vibram soles for traction
4. **Breathability** - Mesh panels for ventilation
5. **Proper Fit** - Room for toe movement, snug heel

### **Popular Models:**
- **Merrell Moab 3 Mid** - Versatile all-around boot
- **Salomon X Ultra 4 Mid** - Lightweight with great grip
- **Keen Targhee III Mid** - Waterproof with ro

╔════════ File Write Operation ═════════╗
║                                       ║
║ Path: hiking_boot_recommendations.txt ║
║ Size: 1152 characters                 ║
║                                       ║
╚═══════════════════════════════════════╝

╔══════════════════════ Write Successful ══════════════════════╗
║ File written successfully to hiking_boot_recommendations.txt ║
╚══════════════════════════════════════════════════════════════╝

Perfect! I've provided you with comprehensive hiking boot recommendations and saved the information to `hiking_boot_recommendations.txt` in your current directory. The file includes brand recommendations, key features to consider, popular models, and important sizing tips to help you make the best choice for your hiking needs.

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

In [24]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you with hiking boot recommendations and then write the response to a file in the current directory.\n\n## Hiking Boot Recommendations\n\nWhen choosing hiking boots, consider these key factors:\n\n### **Top-Rated Hiking Boot Brands:**\n- **Merrell** - Great for beginners, comfortable and affordable\n- **Salomon** - Excellent for technical terrain and durability\n- **Keen** - Known for toe protection and wide fit options\n- **Vasque** - Superior support for heavy loads\n- **La Sportiva** - Premium quality for serious hikers\n\n### **Key Features to Look For:**\n1. **Waterproofing** - Gore-Tex or similar membrane\n2. **Ankle Support** - Mid or high-cut for stability\n3. **Sole Grip** - Vi

In [ ]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

In [ ]:
orchestrator.messages = []

## Congrats!

You've learned how to use agents as tools in Strands Agents to create more complex agentic applications